In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from enum import Enum
from datetime import datetime

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# create an enum for importance
class Importance(Enum):
    NONE = 'None'
    KINDA_IMPORTANT = 'Kinda Important'
    SUPER_IMPORTANT = 'Super Important'
    DATE_DEPENDENT = 'Date Dependent'

# create an enum for overdue
class IfOverdue(Enum):
    MARK_RED_PRIORITIZE = 'Mark Red and Prioritize'
    MARK_RED = 'Mark Red'
    NEVER_MIND = 'Never Mind'



# # create a class for label
# class Label:
#     def __init__(self):
#         self.labels = []

#     def add_label(self, label_name):
#         if label_name not in self.labels:
#             self.labels.append(label_name)

# label_manager = Label()


# testing label
# print(label_manager.labels)
# label_manager.add_label('school')
# label_manager.add_label('work')
# print(label_manager.labels)

In [3]:
# create class Task
class Task:
    def __init__(self, task, is_done=False, label=None, date_due=None, importance=None, is_prioritized=False, is_overdue=False,
                 if_overdue=None, details=None, steps=None, thoughts=None):
        self.task = task
        self.is_done = is_done
        self.label=label

        # # Create an instance of the Label class to manage labels
        # self.label_manager = Label()

        # # Check if label is provided and add it to the label manager
        # if label:
        #     self.label_manager.add_label(label)

        self.date_added = datetime.now()  # Automatically set to the current date and time
        self.date_due = date_due

        # Check if importance is a valid member of the Importance enum
        if importance and importance not in Importance.__members__:
            raise ValueError(f"{importance} is not a valid Importance")
        self.importance = Importance[importance] if importance else None

        self.is_prioritized = is_prioritized
        self.is_overdue = is_overdue

        # Check if if_overdue is a valid member of the IfOverdue enum
        if if_overdue and if_overdue not in IfOverdue.__members__:
            raise ValueError(f"{if_overdue} is not a valid IfOverdue")
        self.if_overdue = IfOverdue[if_overdue] if if_overdue else None        
        
        self.details = details 
        self.steps = steps
        self.thoughts = thoughts

In [4]:
# example
task1 = Task('Complete assignment', label='school', date_due=datetime(2023, 11, 21, 23, 59), importance='SUPER_IMPORTANT', 
             is_prioritized=False, is_overdue=False, if_overdue='MARK_RED_PRIORITIZE', details='30% of the final grade!', 
             steps='1. intro; 2. body; 3. conclusion.', thoughts='this is so hard omg')

task2 = Task('Pick up stones', label='life', importance='KINDA_IMPORTANT', steps='find a stone & pick it up.', thoughts='stones are cool.')

task3 = Task('hackathon', label='school', date_due=datetime(2023, 11, 19, 23, 59), importance='DATE_DEPENDENT', if_overdue='NEVER_MIND',
             details='I like coding.', steps='1. this object; 2. create functions; 3. show it as a dataframe.', thoughts='yessir')


def print_task_details(task):
    for attr, value in task.__dict__.items():
        print(f"{attr}: {value}")


print_task_details(task1)
print()
print_task_details(task3)


task: Complete assignment
is_done: False
label: school
date_added: 2023-11-19 13:28:05.648505
date_due: 2023-11-21 23:59:00
importance: Importance.SUPER_IMPORTANT
is_prioritized: False
is_overdue: False
if_overdue: IfOverdue.MARK_RED_PRIORITIZE
details: 30% of the final grade!
steps: 1. intro; 2. body; 3. conclusion.
thoughts: this is so hard omg

task: hackathon
is_done: False
label: school
date_added: 2023-11-19 13:28:05.648567
date_due: 2023-11-19 23:59:00
importance: Importance.DATE_DEPENDENT
is_prioritized: False
is_overdue: False
if_overdue: IfOverdue.NEVER_MIND
details: I like coding.
steps: 1. this object; 2. create functions; 3. show it as a dataframe.
thoughts: yessir


In [5]:
cols = ['task','done?','label','date added','date due','importance','prioritize it?','is it overdue?','if is overdue','details','steps','thoughts']
'''
task: name of the task
done?: boolean, check if it is done and return True. if True, put the task to database archived_task
label: can customize label e.g. [school, work, life] and choose one to identify the task
date added: (auto generate) the date/time when the task is added
date due: the due date or None 
importance: one of (low, medium, high, time_dependent), where time dependent is relative to date_due
prioritize it?: boolean, pinned to the top if True
is it overdue?: boolean, automatically check if passed the due date but done == False
what if overdue: one of ("mark red and prioritize it", "just mark red", "never mind"). 
    "mark red and prioritize it": turns the entire row to red on the UI & change the prioritize? to True 
    "just mark red": turns the entire row to red on the UI, but do not pin it
    "never mind": put the task to database archived_task (will still have done? == False)
details: string values entered by users if any 
steps: a numbered list of string values entered by users if any
thoughts: string values entered by users if any 
'''
df = pd.DataFrame(
    columns = cols
)
print(df.shape)
df

(0, 12)


,task,done?,label,date added,date due,importance,prioritize it?,is it overdue?,if is overdue,details,steps,thoughts


In [6]:
# TODO: let user input the information of task(s)
# which creates a Task() instance

In [7]:
# link a Task() instance to df

def get_instance_info(task):
    task_values = []
    for attr, value in task.__dict__.items():
        if isinstance(value, Enum):
            value = value.value
            # print(value)
        task_values.append(value)
    return task_values

def add_instance_to_df(df, task):
    task_values = get_instance_info(task)
    df.loc[len(df)] = task_values
    return df


In [8]:
# tests
get_instance_info(task1)
add_instance_to_df(df,task1)
add_instance_to_df(df,task3)
df.drop_duplicates(inplace=True)
df

,task,done?,label,date added,date due,importance,prioritize it?,is it overdue?,if is overdue,details,steps,thoughts
0,Complete assignment,False,school,2023-11-19 13:28:05.648505,2023-11-21 23:59:00,Super Important,False,False,Mark Red and Prioritize,30% of the final grade!,1. intro; 2. body; 3. conclusion.,this is so hard omg
1,hackathon,False,school,2023-11-19 13:28:05.648567,2023-11-19 23:59:00,Date Dependent,False,False,Never Mind,I like coding.,1. this object; 2. create functions; 3. show i...,yessir


In [9]:
# create importance score 
# use ordinal encoder?

def create_importance_score(df):
    '''
    create importance score (with a range of 0 to 7)
    if it is date dependent, give it a score of 7 - (days in between)
    (this means if it is due in less than 1 day, it will have a score of 7)
    if it is super important, give it a score 6.5 (while it will still be less important than those due in less than 1 day)
    if it is kinda important, give it a score 3.5 (date dependent will result higher if it is due in three days)
    '''
    importance_score = []
    for index, row in df.iterrows():
        if row['importance']=='Date Dependent':
            time_left = row['date due'] - datetime.now()
            days_left = time_left.days
            score = max(0,7-days_left)
            importance_score.append(score)
        elif row['importance']=='Super Important':
            importance_score.append(6.5)
        elif row['importance']=='Kinda Important':
            importance_score.append(3.5)

    df_scored = df.copy()

    df_scored['importance_score']=importance_score
    return df_scored

print(create_importance_score(df).shape)
create_importance_score(df)


(2, 13)


,task,done?,label,date added,date due,importance,prioritize it?,is it overdue?,if is overdue,details,steps,thoughts,importance_score
0,Complete assignment,False,school,2023-11-19 13:28:05.648505,2023-11-21 23:59:00,Super Important,False,False,Mark Red and Prioritize,30% of the final grade!,1. intro; 2. body; 3. conclusion.,this is so hard omg,6.5
1,hackathon,False,school,2023-11-19 13:28:05.648567,2023-11-19 23:59:00,Date Dependent,False,False,Never Mind,I like coding.,1. this object; 2. create functions; 3. show i...,yessir,7.0


In [10]:
print(df.shape)
df

(2, 12)


,task,done?,label,date added,date due,importance,prioritize it?,is it overdue?,if is overdue,details,steps,thoughts
0,Complete assignment,False,school,2023-11-19 13:28:05.648505,2023-11-21 23:59:00,Super Important,False,False,Mark Red and Prioritize,30% of the final grade!,1. intro; 2. body; 3. conclusion.,this is so hard omg
1,hackathon,False,school,2023-11-19 13:28:05.648567,2023-11-19 23:59:00,Date Dependent,False,False,Never Mind,I like coding.,1. this object; 2. create functions; 3. show i...,yessir
